In [ ]:
from utils.segment_hota.map_points_to_segments import map_points_to_segments
from utils.segment_hota.detection_accuracy import detection_accuracy
from utils.segment_hota.association_accuracy import association_accuracy
from utils.segment_hota.types import Polygon, Segment, SegmentDetection
from utils.segment_hota.object_type import object_type

In [ ]:
import os
import json
import shapely.wkt
import shapely.wkt
import postgis
import postgis.geometry

In [ ]:
with open(os.path.join('/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Mini/annotation_splitted.json'), 'r') as f:
    ground_truth_split = json.load(f)
    

In [ ]:
with open(os.path.join('/data/chanwutk/code/apperception/outputs/segment-tracking-without-de-segment.json'), 'r') as f:
    prediction_split = json.load(f)
# [*prediction_split.values()][0]



In [ ]:
[*prediction_split.keys()][0]

In [ ]:
with open(os.path.join('/data/chanwutk/code/apperception/outputs/segment-tracking-without-de-frame-id.json'), 'r') as f:
    prediction_frame_id_split = json.load(f)
# [*prediction_frame_id_split.values()][0]



In [ ]:
[*ground_truth_split.values()][30][1]

In [ ]:
[*ground_truth_split.keys()][0]

In [ ]:
def format_prediction(prediction_frames, predictions):
    out = []
    for fid, pds in zip(prediction_frames, predictions):
        for oid, pd in pds.items():
            road_polygon_info = pd['road_polygon_info']
            if road_polygon_info is None:
                continue
#             print(pd['segment_line'])
#             print(fid)
            out.append(SegmentDetection(
                oid=oid,
                did=str(pd['detection_id']),
                fid=fid,
                timestamp=pd['timestamp'],
                segment=Segment(
                    id=pd['segment_line'],
                    line=shapely.wkb.loads(pd['segment_line'], hex=True),
#                     line='',
                    polygon=Polygon(
                        id=road_polygon_info['id'],
                        polygon=shapely.wkt.loads(road_polygon_info['polygon']),
                        segments=None,
                    ),
                ),
                location='boston-seaport',
                type=pd['type']
            ))
    return out

In [ ]:
for name, (sdts, groundtruths) in ground_truth_split.items():
    if len(groundtruths) == 0:
        continue

    locations = set(a['location'] for a in groundtruths)
    assert len(locations) == 1, locations
    if [*locations][0] != 'boston-seaport':
        continue
    
    pd_keys = name.split('_')
    pd_key = f"{pd_keys[0]}-{'_'.join(pd_keys[1:])}"
    
    if pd_key not in prediction_split:
        continue

    predictions = prediction_split[pd_key]
    prediction_frames = prediction_frame_id_split[pd_key]

    formatted_predictions = format_prediction(prediction_frames, predictions)
    formatted_groundtruths = map_points_to_segments(groundtruths)
    
    formatted_groundtruths = [
        gt
        for gt
        in formatted_groundtruths
        if gt.type.startswith('vehicle') and object_type(gt.type) is not None
    ]
    for gt in formatted_groundtruths:
        t = object_type(gt.type)
        assert t is not None, gt.type
        gt.type = t
    
    print(name)
    print(len(formatted_groundtruths), len(groundtruths))
    print('DetA', detection_accuracy  (formatted_predictions, formatted_groundtruths   ) * 100, '%')
    print('AssA', association_accuracy(formatted_predictions, formatted_groundtruths, 0) * 100, '%')
    print()
    print()